In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('datos/df_total.csv',encoding='utf-8')

In [3]:
df

,url,news,Type
0,https://www.larepublica.co/redirect/post/3201905,Durante el foro La banca articulador empresari...,Otra
1,https://www.larepublica.co/redirect/post/3210288,El regulador de valores de China dijo el domin...,Regulaciones
2,https://www.larepublica.co/redirect/post/3240676,En una industria históricamente masculina como...,Alianzas
3,https://www.larepublica.co/redirect/post/3342889,Con el dato de marzo el IPC interanual encaden...,Macroeconomia
4,https://www.larepublica.co/redirect/post/3427208,Ayer en Cartagena se dio inicio a la versión n...,Otra
...,...,...,...
1212,https://www.bbva.com/es/como-lograr-que-los-in...,En la vida de toda empresa emergente llega un ...,Innovacion
1213,https://www.bbva.com/es/podcast-como-nos-afect...,La espiral alcista de los precios continúa y g...,Macroeconomia
1214,https://www.larepublica.co/redirect/post/3253735,Las grandes derrotas nacionales son experienci...,Alianzas
1215,https://www.bbva.com/es/bbva-y-barcelona-healt...,BBVA ha alcanzado un acuerdo de colaboración c...,Innovacion


In [4]:
print(df['Type'].value_counts())

Type
Macroeconomia     340
Alianzas          247
Innovacion        195
Regulaciones      142
Sostenibilidad    137
Otra              130
Reputacion         26
Name: count, dtype: int64


In [7]:
from sklearn.model_selection import train_test_split
# crear un conjunto de datos de entrenamiento y prueba




X_train, X_test, y_train, y_test = train_test_split(df['news'], df['Type'], test_size=0.2,random_state=0)

In [41]:
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
CountVectorizer = CountVectorizer()
X_train_transformed = CountVectorizer.fit_transform(X_train)
X_test_transformed = CountVectorizer.transform(X_test)

In [10]:
X_train_transformed

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 232655 stored elements and shape (973, 26417)>

In [12]:
# solo para mostrar como queda el vector

X_train_transformed_dense = X_train_transformed.toarray()
print(X_train_transformed_dense)

[[0 2 0 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [13]:
# guardar X_train_transformed  a csv

df_X_train_transformed = pd.DataFrame(X_train_transformed_dense)
df_X_train_transformed.to_csv('datos/X_train_transformed.csv', index=False)


In [14]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

# crear el modelo
model = MultinomialNB()
model.fit(X_train_transformed, y_train)

# predecir
y_pred = model.predict(X_test_transformed)

# calcular la exactitud
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %.2f' % (accuracy*100))


Accuracy: 78.28


## Stemmeng

In [15]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer

In [16]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /home/codespace/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/codespace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [17]:
stemmer = SnowballStemmer('spanish')

In [18]:
def clean_text(text):
    text = text.lower()
    text = word_tokenize(text)
    text = [stemmer.stem(word) for word in text if word not in stopwords.words('spanish')]
    text = ' '.join(text)
    return text

In [21]:
def tokenize_and_stem(text):
    tokens = word_tokenize(text.lower())
    stems = [stemmer.stem(token) for token in tokens if token.isalpha()]
    return ' '.join(stems)

In [22]:
# aplicacamos la funciion de tokenizacion y stemming

df['news_stemmer'] = df['news'].apply(tokenize_and_stem)

In [23]:
df['news_stemmer']

0       durant el for la banc articul empresarial par ...
1       el regul de valor de chin dij el doming que bu...
2       en una industri histor masculin com lo es la a...
3       con el dat de marz el ipc interanual encaden s...
4       ayer en cartagen se dio inici a la version num...
                              ...                        
1212    en la vid de tod empres emergent lleg un momen...
1213    la espiral alcist de los preci continu y gener...
1214    las grand derrot nacional son experient trauma...
1215    bbva ha alcanz un acuerd de colabor con barcel...
1216    casi entrand a la part final de noviembr la ep...
Name: news_stemmer, Length: 1217, dtype: object

In [26]:
X=df['news_stemmer']
y=df['Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
CountVectorizer = CountVectorizer()
X_train_transformed = CountVectorizer.fit_transform(X_train)
X_test_transformed = CountVectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_transformed, y_train)

y_pred = model.predict(X_test_transformed)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %.2f' % (accuracy*100))



Accuracy: 79.10


In [27]:
X_train_transformed

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 196898 stored elements and shape (973, 11819)>

In [29]:
# guardar X_train_transformed  a csv

df_X_train_transformed = pd.DataFrame(X_train_transformed_dense)
df_X_train_transformed.to_csv('datos/X_train_transformed.csv', index=False)

## lematizacion

In [30]:
import spacy

In [35]:
nlp=spacy.load('es_core_news_sm')


In [36]:
def lemmatize_text(text):
    doc = nlp(text.lower())
    lemmas = [token.lemma_ for token in doc if token.is_alpha]
    text = ' '.join(lemmas)
    return text

In [37]:
df['news_lemmatized'] = df['news'].apply(lemmatize_text)


In [38]:
df['news_lemmatized']

0       durante el foro el banca articulador empresari...
1       el regulador de valor de china decir el doming...
2       en uno industria históricamente masculino como...
3       con el dato de marzo el ipc interanual encaden...
4       ayer en cartagena él dar inicio a el versión n...
                              ...                        
1212    en el vida de todo empresa emergente llegar un...
1213    el espiral alcista de el precio continuar y ge...
1214    el grande derrota nacional ser experiencia tra...
1215    bbva haber alcanzar uno acuerdo de colaboració...
1216    casi entrar a el parte final de noviembre el é...
Name: news_lemmatized, Length: 1217, dtype: object

In [39]:
df['news_lemmatized'][3]

'con el dato de marzo el ipc interanual encadenar su decimoquinto tasa positivo consecutivo el inflación publicado por el ine él haber mantener igual respecto al avance del de marzo y él situar punto por encima del dato de febrero que ascender al ese punto de diferencia el mayor parte él colocar el grupo de el vivienda punto por el subida de el electricidad y el del transporte punto por el alza de el carburante también impulsar el ipc de marzo el aumento de el precio de el restauración y el servicio de alojamiento y al encarecimiento generalizado de el alimento especialmente del pescado y el marisco de el carne de el legumbr y hortaliza y de el leche el queso y el tener en cuenta el rebaja del impuesto especial sobre el electricidad y el variación sobre otro impuesto el ipc interanual alcanzar en marzo nueve décima más que el tasa general del así él reflejar el ipc a impuesto constante que el ine también publicar en el marco de este inflación subyacente sin alimento no elaborado ni pro

In [42]:
X=df['news_lemmatized']
y=df['Type']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=0)
CountVectorizer = CountVectorizer()
X_train_transformed = CountVectorizer.fit_transform(X_train)
X_test_transformed = CountVectorizer.transform(X_test)

model = MultinomialNB()
model.fit(X_train_transformed, y_train)

y_pred = model.predict(X_test_transformed)

accuracy = accuracy_score(y_test, y_pred)
print('Accuracy: %.2f' % (accuracy*100))

Accuracy: 77.87


In [43]:
X_train_transformed

<Compressed Sparse Row sparse matrix of dtype 'int64'
	with 191810 stored elements and shape (973, 16424)>